# 15강. 사람의 몸짓을 읽어보자
## 15-6. 코드로 이해하는 pose estimation
### SimpleBaseline - tf2

In [1]:
import os

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

resnet = tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet')

94781440/94765736 [==============================] - 1s 0us/step


In [2]:
# deconv module 만들기
upconv1 = tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same')
bn1 = tf.keras.layers.BatchNormalization()
relu1 = tf.keras.layers.ReLU()
upconv2 = tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same')
bn2 = tf.keras.layers.BatchNormalization()
relu2 = tf.keras.layers.ReLU()
upconv3 = tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same')
bn3 = tf.keras.layers.BatchNormalization()
relu3 = tf.keras.layers.ReLU()

In [3]:
# deconv module 만들기(중복 제거)
def _make_deconv_layer(num_deconv_layers):
    seq_model = tf.keras.models.Sequential()
    for i in range(num_deconv_layers):
        seq_model.add(tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same'))
        seq_model.add(tf.keras.layers.BatchNormalization())
        seq_model.add(tf.keras.layers.ReLU())
    return seq_model

upconv = _make_deconv_layer(3)

In [4]:
final_layer = tf.keras.layers.Conv2D(17, kernel_size=(1,1), padding='same')

In [5]:
# 각각의 요소를 합쳐 모델을 완성하기
inputs = keras.Input(shape=(256, 192, 3))
x = resnet(inputs)
x = upconv(x)
out = final_layer(x)
model = keras.Model(inputs, out)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 192, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, None, None, 2048)  23587712  
_________________________________________________________________
sequential (Sequential)      (None, 64, 48, 256)       10489600  
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 48, 17)        4369      
Total params: 34,081,681
Trainable params: 34,027,025
Non-trainable params: 54,656
_________________________________________________________________


In [6]:
# 가상의 이미지를 넣어서 출력이 잘 나오는지 확인하기
np_input = np.zeros((1,256,192,3), dtype=np.float32)
tf_input = tf.convert_to_tensor(np_input, dtype=tf.float32)
print('input shape')
print (tf_input.shape)
print('\n')

tf_output = model(tf_input)
print('output shape')
print (tf_output.shape)
print (tf_output[0,:10,:10,:10])

input shape
(1, 256, 192, 3)


output shape
(1, 64, 48, 17)
tf.Tensor(
[[[-3.96982906e-03  2.20121443e-03 -8.52179900e-03  1.29217777e-04
   -2.89635896e-03 -8.00100993e-03 -3.59922042e-03  3.71248141e-04
   -2.32576230e-03  1.31047238e-03]
  [-1.04267206e-02  9.24294908e-03 -8.99320189e-03 -1.79555602e-02
    7.79469823e-03  2.08173390e-03  1.71510484e-02  6.31922996e-03
    2.63131480e-03 -8.21489096e-03]
  [ 6.64827367e-03  4.83499520e-04 -9.74830985e-03 -1.94180151e-03
   -5.16201183e-03  1.53928529e-03  6.90780208e-03 -7.27311615e-03
   -1.26691591e-02 -2.10808776e-03]
  [-2.60492321e-03 -5.85054187e-03 -2.13629566e-02 -7.62721011e-03
   -4.14571114e-04 -1.71389654e-02  1.05734104e-02  8.47075135e-03
   -8.01410060e-03 -9.18843132e-03]
  [ 4.20670444e-03  1.09382272e-02 -9.09828022e-03  7.76937604e-03
   -1.05111087e-02  1.25560621e-02  1.09534161e-02 -1.06445896e-05
   -1.44990552e-02 -2.05670949e-03]
  [-2.06606854e-02  2.75893118e-02 -4.21069301e-04 -2.15376280e-02
   -6.151896

- label 을 어떻게 만드는지, 48x64 의 출력을 어떻게 사용해야 하는지는 16강에서!